In [40]:
%pip install pandas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [41]:
%pip install datasets transformers torch evaluate 'transformers[torch]' accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [42]:
import pandas as pd



In [43]:
data = pd.read_csv('tinybert_model.csv',index_col=0)

In [44]:
data

# sagemaker => airflow => 모델 학습 및 배포

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [45]:
# 데이터셋의 형태 변환

from datasets import Dataset

dataset = Dataset.from_pandas(data)
dataset = dataset.train_test_split(test_size=0.3)
dataset


DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment', '__index_level_0__'],
        num_rows: 35000
    })
    test: Dataset({
        features: ['review', 'sentiment', '__index_level_0__'],
        num_rows: 15000
    })
})

In [46]:
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [60]:
label2id = {'negative':0, 'positive':1} # 레이블 문자열 데이터를 숫자로 매핑 딕셔너리

dataset = dataset.map(lambda x: {'label': label2id[x['sentiment']]}) # 레이블 문자열 데이터를 숫자로 매핑

dataset
# def preprocess_function(examples):
#     return tokenizer(examples['text'], truncation=True)

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 35000
    })
    test: Dataset({
        features: ['review', 'sentiment', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 15000
    })
})

In [61]:
dataset['train'][0]


{'review': "Love hurts. That, I think, is the main message Mike Binder's newest film Reign Over Me brings across. Whether that love has caused your relationship to become stagnant, or has brought anger from the one you love cheating for years, or has broken your heart to the point of being unable to open yourself up to the world, love hurts. The great thing about this film, however, is not in its portrayal of these lost souls trying to let their past heartbreaks go, but in the eventual restart of new bonds for the future. No one in this drama is perfect; they are all at some degree trapped emotionally in relationships that they can't free themselves from alone. There is some heavy subject material here and I credit Binder for never making the story turn into a political diatribe, but instead infusing the serious moments with some real nice comedic bits allowing the tale to stay character-based and small in scale compared to the epic event that looms overhead. What could have become a t

In [62]:
%pip install transformers torch # 모델 불러오기 위해 필요

Python(52197) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [63]:
# 데이터 준비가 끝났습니다. => 모델 불러오기

from transformers import AutoTokenizer #GPU
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Hugginc Face에서 tinybert 검색 -> huawei-noah 모델 선택
model = 'huawei-noah/TinyBERT_General_4L_312D'
tokenizer = AutoTokenizer.from_pretrained(model, use_fast=True)
tokenizer



BertTokenizerFast(name_or_path='huawei-noah/TinyBERT_General_4L_312D', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [64]:
tokenizer("Python is easy")

tokenizer(dataset['train'][0]['review'])

{'input_ids': [101, 2293, 13403, 1012, 2008, 1010, 1045, 2228, 1010, 2003, 1996, 2364, 4471, 3505, 14187, 2121, 1005, 1055, 14751, 2143, 5853, 2058, 2033, 7545, 2408, 1012, 3251, 2008, 2293, 2038, 3303, 2115, 3276, 2000, 2468, 2358, 8490, 16885, 1010, 2030, 2038, 2716, 4963, 2013, 1996, 2028, 2017, 2293, 16789, 2005, 2086, 1010, 2030, 2038, 3714, 2115, 2540, 2000, 1996, 2391, 1997, 2108, 4039, 2000, 2330, 4426, 2039, 2000, 1996, 2088, 1010, 2293, 13403, 1012, 1996, 2307, 2518, 2055, 2023, 2143, 1010, 2174, 1010, 2003, 2025, 1999, 2049, 13954, 1997, 2122, 2439, 9293, 2667, 2000, 2292, 2037, 2627, 27724, 2015, 2175, 1010, 2021, 1999, 1996, 9523, 23818, 1997, 2047, 9547, 2005, 1996, 2925, 1012, 2053, 2028, 1999, 2023, 3689, 2003, 3819, 1025, 2027, 2024, 2035, 2012, 2070, 3014, 7567, 14868, 1999, 6550, 2008, 2027, 2064, 1005, 1056, 2489, 3209, 2013, 2894, 1012, 2045, 2003, 2070, 3082, 3395, 3430, 2182, 1998, 1045, 4923, 14187, 2121, 2005, 2196, 2437, 1996, 2466, 2735, 2046, 1037, 2576, 229

In [65]:
dataset['train'][0]['review']

"Love hurts. That, I think, is the main message Mike Binder's newest film Reign Over Me brings across. Whether that love has caused your relationship to become stagnant, or has brought anger from the one you love cheating for years, or has broken your heart to the point of being unable to open yourself up to the world, love hurts. The great thing about this film, however, is not in its portrayal of these lost souls trying to let their past heartbreaks go, but in the eventual restart of new bonds for the future. No one in this drama is perfect; they are all at some degree trapped emotionally in relationships that they can't free themselves from alone. There is some heavy subject material here and I credit Binder for never making the story turn into a political diatribe, but instead infusing the serious moments with some real nice comedic bits allowing the tale to stay character-based and small in scale compared to the epic event that looms overhead. What could have become a trite vehicl

In [66]:
def tokenize(batch):  # 토큰화 함수 정의
    # padding=True => 텍스트 데이터를 동일한 길이로 패딩 처리
    # truncation=True => 맥스 토큰 갯수를 넘어가는 문장은 잘라내겠습니다.
    # max_length=300 => 토큰화된 결과의 최대 길이를 300개 (300개 이상의 토큰은 자른다. 300개 이하는 패딩처리)
    temp = tokenizer(batch['review'], padding=True, truncation=True, max_length=300)

    return temp

In [67]:
dataset = dataset.map(tokenize, batched=True, batch_size=None)
dataset

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 35000
    })
    test: Dataset({
        features: ['review', 'sentiment', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 15000
    })
})

In [68]:
dataset['train'][0]

{'review': "Love hurts. That, I think, is the main message Mike Binder's newest film Reign Over Me brings across. Whether that love has caused your relationship to become stagnant, or has brought anger from the one you love cheating for years, or has broken your heart to the point of being unable to open yourself up to the world, love hurts. The great thing about this film, however, is not in its portrayal of these lost souls trying to let their past heartbreaks go, but in the eventual restart of new bonds for the future. No one in this drama is perfect; they are all at some degree trapped emotionally in relationships that they can't free themselves from alone. There is some heavy subject material here and I credit Binder for never making the story turn into a political diatribe, but instead infusing the serious moments with some real nice comedic bits allowing the tale to stay character-based and small in scale compared to the epic event that looms overhead. What could have become a t

In [69]:
from transformers import AutoModelForSequenceClassification  

id2label = {0:'negative', 1:'positive'}

model = AutoModelForSequenceClassification.from_pretrained(
    model, #  사전 학습된 모델의 이름
    num_labels=len(label2id), # 레이블 갯수
    label2id=label2id, # 레이블 문자열 데이터를 숫자로 매핑 딕셔너리
    id2label=id2label
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
%pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [70]:
import evaluate
import numpy as np
# Hugging Face evaluate 라이브러리에서 accuracy(정확도) 메트릭을 불러옵니다.
accuracy = evaluate.load('accuracy')
# compute_metrics 함수는 모델의 예측값과 실제 레이블을 받아 정확도를 계산합니다.
def compute_metrics(eval_pred):  #평가 지표 계산 함수 정의
    # eval_pred는 모델의 예측값과 실제 레이블을 포함하는 튜플입니다.
    predictions, labels = eval_pred  # predictions: 모델이 예측한 값, labels: 실제 정답 레이블
    
    # predictions는 모델이 출력한 확률값 배열이며, 가장 높은 확률을 가진 클래스를 선택합니다.
    # np.argmax는 각 샘플에 대해 가장 높은 확률값을 가진 클래스의 인덱스를 반환합니다.
    predictions = np.argmax(predictions, axis=1)  # 각 샘플별로 가장 높은 확률을 가진 클래스 선택
    
    # accuracy.compute는 예측값(predictions)과 실제값(labels)을 비교하여 정확도를 계산합니다.
    # predictions: 모델이 예측한 클래스, references=labels: 실제 정답 클래스
    return accuracy.compute(predictions=predictions, references=labels) #예측값과 실제값 비교 후 정확도 계산

In [19]:
# # !pip install tf-keras
# %pip install 'transformers[torch]'
# %pip install accelerate -U

# # pip freeze

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [71]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir='train_dir', # 학습 결과를 저장할 디렉토리
    overwrite_output_dir=True,
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=32, # 각 디바이스당 학습 배치 크기
    per_device_eval_batch_size=32, # 각 디바이스당 평가 배치 크기
    eval_strategy='epoch' # 매 에포크마다 평가
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics, # 평가 지표를 계산하 함수
    tokenizer=tokenizer # 텍스트를 토큰으로 변환하는 도구
)

/var/folders/py/91q9ql1j6412wg5tjn0xry080000gn/T/ipykernel_52103/1138123528.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [72]:
trainer.train()

  0%|          | 0/3282 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
trainer.evaluate()  # 데이터 정확도 확인으로 평가 

In [ ]:
trainer.save_model('tinybert_sentiment_analysis')  # 옆에 새로운 디렉토리에 모델이 저장될 것임

In [ ]:
from transformers import pipeline

classifier = pipeline('text_classification', model='tinybert_sentiment_analysis', device=device)
classifier



In [ ]:
data = ['This is crazy']
classifier(data)

In [ ]:
vader, nltk, textblob